In [4]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [5]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

In [28]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [29]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 84


In [30]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Borough,Neighbourhood,Postcode
0,Central Toronto,Lawrence Park,M4N
1,Scarborough,Steeles West,M1W
2,Downtown Toronto,St. James Town,M5C
3,North York,"Northwood Park, York University",M3J
4,Central Toronto,"The Annex, Yorkville",M5R
5,Scarborough,"Agincourt North, Milliken",M1V
6,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T
7,Downtown Toronto,University of Toronto,M5S
8,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K
9,East York,Woodbine Heights,M4C


In [9]:
df_toronto.shape

(84, 3)

In [13]:
import geocoder

In [20]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [24]:
codes=pd.read_csv('Geospatial_Coordinates.csv')
codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
#houston_housing=houston_grouped.join(SUM_TotHousing.set_index('SNBNAME'), on='SNBNAME')
#df_toronto.shape
#SUM_TotHousing.rename(columns={'Name':'SNBNAME'},inplace=True)
df_toronto.rename(columns={'Postcode':'Postal Code'},inplace=True)
df_toronto.head()

,Borough,Neighbourhood,Postal Code
0,Central Toronto,Lawrence Park,M4N
1,Scarborough,Steeles West,M1W
2,Downtown Toronto,St. James Town,M5C
3,North York,"Northwood Park, York University",M3J
4,Central Toronto,"The Annex, Yorkville",M5R


In [35]:
toronto_codes=df_toronto.join(codes.set_index('Postal Code'), on='Postal Code')

In [36]:
toronto_codes.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
1,Scarborough,Steeles West,M1W,43.799525,-79.318389
2,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
3,North York,"Northwood Park, York University",M3J,43.767980,-79.487262
4,Central Toronto,"The Annex, Yorkville",M5R,43.672710,-79.405678


In [37]:
columnsTitles=['Postal Code','Borough','Neighbourhood','Latitude','Longitude']
toronto_codes=toronto_codes.reindex(columns=columnsTitles)

In [38]:
toronto_codes.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M1W,Scarborough,Steeles West,43.799525,-79.318389
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
4,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678
